### Multi Order Spoofing Alert Event Analysis

In [1]:
import os; os.chdir('/Users/sqian/MKTSRV')
import pandas as pd; pd.set_option('mode.chained_assignment',None)
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import matplotlib as mpl
import re
import xlrd
from xlsxwriter.utility import xl_rowcol_to_cell

from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty, littleQA, totalUP
from src.data_models.SmartsDataModel import SmartsDataModel
from src.data_models.SmartsCsvDataModel import SmartsCsvDataModel
from src.utility.DataModelUtility import execute_query_data_frame

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### 1) Read in SMARTS Alerts

In [6]:
alerts0 = fixNum(unSMART(pd.read_excel('~/Documents/vmfldr/smarts1_2019Aug.xlsx')))

-> type fixed, kept 179 rows, 12 columns


In [8]:
alerts0.dtypes

Date              datetime64[ns]
Time                      object
InstrumentCode            object
ShortText                 object
LongText                  object
Datetime          datetime64[ns]
ShortText_mod             object
AlertCode                  int64
AccountIDName              int64
ReissueCount               int64
month                      int64
year                       int64
dtype: object

#### 2) Rolled Up Text File Alerts (4032-eod)

In [20]:
alerts4032_eod = alerts0.loc[alerts0['ShortText'] == 'MULTI ORDER SPOOFING EOD',['Date','InstrumentCode','AccountIDName','ShortText']].reset_index(drop=True)
alerts4032_eod2 = pd.DataFrame(alerts4032_eod.groupby(['Date','InstrumentCode','AccountIDName']).count().reset_index(drop=False))
alerts4032_eod2['Date2'] = '_' + alerts4032_eod2['Date'].astype(str).str.replace('-','') + '.csv'
alerts4032_eod2['ImportSyntax'] = '~/Documents/vmfldr/spoof/4032_' + alerts4032_eod2['InstrumentCode'] + '_Broker_' + alerts4032_eod2['AccountIDName'].astype(str) + alerts4032_eod2['Date2']

events4032_eod = pd.DataFrame()
for i in range(0,alerts4032_eod2.shape[0]): 
    temp = pd.read_csv(alerts4032_eod2.loc[i,'ImportSyntax'])
    temp['Date'] = alerts4032_eod2.loc[i,'Date']
    events4032_eod = events4032_eod.append(temp)
    
events4032_eod.columns = events4032_eod.columns.str.replace(' ','').str.replace('Spoofing','spoof').str.replace('GenuineSide','gen').str.replace('side','')

#### 3) Spoof side cleaning (end with 51,685 records)

In [21]:
data0 = pd.DataFrame(events4032_eod)
data0['spoof_entry'] = pd.to_datetime(data0['Date'].astype(str) + '' + data0['Entrytimespoof'].fillna(' 23:59:59.999'))
data0['spoof_exit'] = pd.to_datetime(data0['Date'].astype(str) + '' + data0['Amend/deletetimespoofSide'].fillna(' 00:00:00.000'))
data0['spoof_vol'] = data0['VolumespoofSide'].astype(str).str.replace('x','').astype(float)
data0['spoof_price'] = data0['PricespoofSide'].astype(str).str.replace(' ','').astype(float)
data0['spoofOrderCount'] = data0['spoofOrderCount'].fillna(0)

In [22]:
data1 = data0.rename(columns={'Security':'symbol','Broker':'acc'}).reset_index(drop=True)
data1['gen_place'] = pd.to_datetime(data1['Date'].astype(str) + '' + data1['Entry/amendtimegen'].fillna(' 00:00:00.000'))
data1['gen_fill'] = pd.to_datetime(data1['Date'].astype(str) + '' + data1['Tradetimegen'].fillna(' 00:00:00.000'))
data1['gen_vol'] = data1['Volumegen'].astype(str).str.replace('x','').astype(float)
data1['gen_price0'] = data1['Bestpricebeforespoofing'].astype(str).str.replace(' ','').astype(float)
data1['gen_price1'] = data1['Tradeprice'].astype(str).str.replace(' ','').astype(float)

In [24]:
columns2keep = ['Entrytimespoof', 'Amend/deletetimespoofSide', 'VolumespoofSide', 'PricespoofSide', 
                'Entry/amendtimegen', 'Tradetimegen', 'Volumegen', 'Bestpricebeforespoofing', 'Tradeprice',
                'Genuine/spoofRatio', 'Gainthistrade', 'Totalgain']
data2 = data1.drop(columns=columns2keep).reset_index(drop=True).reset_index(drop=False)
data2['spoof_seconds'] = (data2['spoof_price'] / data2['spoof_price']) * (data2['spoof_exit'] - data2['spoof_entry']).dt.total_seconds()

data2.to_csv('~/Desktop/2019June_SpoofingRollup.csv')

#### 5) Group by

In [25]:
data2gbm2 = data2.groupby(['symbol', 'acc','Date', 'spoofEventNr'])

method2_a = data2gbm2[['spoof_entry']].min().add_suffix('_min')
method2_b = data2gbm2[['gen_place', 'spoof_exit', 'gen_fill', 'gen_price0', 'gen_price1']].max().add_suffix('_max')
method2_part1 = pd.concat([method2_a, method2_b], axis=1)#.reset_index()

method2_part2 = method2_part1[['spoof_entry_min', 'gen_place_max', 'spoof_exit_max', 'gen_fill_max']]
method2 = pd.concat([method2_part2.min(axis=1), method2_part2.max(axis=1)], axis=1).rename(columns={0:'dt_min',1:'dt_max'}).reset_index().sort_values(['symbol','acc','Date','spoofEventNr'])
method2.head()

# method2remaining = method2.loc[~method2.index.isin(method1dispose.index.tolist())]
# method2remaining

,symbol,acc,Date,spoofEventNr,dt_min,dt_max
0,BTCUSD,22440,2019-08-01,1,2019-08-01 02:54:17.793,2019-08-01 02:54:22.596
1,BTCUSD,22440,2019-08-01,2,2019-08-01 02:54:17.793,2019-08-01 02:54:22.674
2,BTCUSD,22440,2019-08-01,3,2019-08-01 02:54:17.793,2019-08-01 02:54:22.674
3,BTCUSD,22440,2019-08-01,4,2019-08-01 02:54:17.793,2019-08-01 02:54:22.674
4,BTCUSD,22440,2019-08-01,5,2019-08-01 02:54:17.793,2019-08-01 02:54:22.767


#### > By Date/Symbol time ranges

In [26]:
data2gbm3 = data2.groupby(['symbol','Date'])

method2_datesymbol = pd.concat([data2gbm3['spoof_entry'].min(),data2gbm3[['spoof_exit','gen_fill']].max().max(axis=1)],axis=1).rename(columns={'spoof_entry':'dtmin',0:'dtmax'}).reset_index()
method2_datesymbol.head()

,symbol,Date,dtmin,dtmax
0,BTCUSD,2019-08-01,2019-08-01 02:54:17.793,2019-08-01 16:14:57.847
1,BTCUSD,2019-08-02,2019-08-02 12:19:03.984,2019-08-02 22:37:46.998
2,BTCUSD,2019-08-04,2019-08-04 18:36:20.491,2019-08-04 18:36:25.406
3,BTCUSD,2019-08-05,2019-08-05 18:24:31.210,2019-08-05 21:33:50.185
4,BTCUSD,2019-08-06,2019-08-06 03:05:16.275,2019-08-06 20:37:15.363


In [27]:
method2_datesymbol['dt_min'] = method2_datesymbol['dtmin'].astype(str)
method2_datesymbol['dt_max'] = method2_datesymbol['dtmax'].astype(str)
method2_datesymbol.shape

(34, 6)

In [28]:
record = method2_datesymbol.iloc[0]

In [ ]:
queryTEMPLATE = """
    SELECT * from ms_dev.daily_best_prices_split_by_account 
    WHERE event_date::date='{date}' and symbol='{symbol}' and date_time BETWEEN '{t1}' AND '{t2}' 
    """.format(date = record['Date'].date(), symbol = record['symbol'], t1 = record['dt_min'], t2 = record['dt_max'])

dbp = execute_query_data_frame(queryTEMPLATE, 'gemrdsdb', ssh = 'interim'); test

In [19]:
dbp.to_pickle('~/Desktop/dpb20190701.pkl')

In [21]:
queryTEMPLATE = """
    SELECT *
    FROM ms_prod.smarts_raw_data
    WHERE data_from_date = '{date}' and symbol = '{symbol}' and (event_date + event_time::time + event_millis::time) BETWEEN '{t1}' AND '{t2}' AND event_type in ('Place','Fill', 'Cancel') 
    """.format(date = record['Date'].date(), symbol = record['symbol'], t1 = record['dt_min'], t2 = record['dt_max'])

srd = execute_query_data_frame(queryTEMPLATE, 'gemrdsdb', ssh = 'interim')     

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:00.511870


INFO:root:Loaded dataframe with 994110 rows in 0:05:51.173570.


In [22]:
srd.to_pickle('~/Desktop/srd20190701.pkl')

#### > Gathering filled records from DB that contains all relevant events

In [ ]:
queryTEMPLATE = """
    SELECT *
    FROM ms_prod.smarts_raw_data
    WHERE data_from_date = '{date}' and symbol = '{symbol}' and (event_date + event_time::time + event_millis::time) BETWEEN '{t1}' AND '{t2}' AND event_type in ('Place','Fill') 
    """
                     
method2_filldf = pd.DataFrame()
for i in range(0,method2_datesymbol.shape[0]):
    record = method2_datesymbol.iloc[i]
    test = execute_query_data_frame(queryTEMPLATE.format(date = record['Date'].date(), symbol = record['symbol'], t1 = record['dt_min'], t2 = record['dt_max']), 'gemrdsdb', ssh = 'interim')     
    print(i, test.shape)
    method2_filldf = method2_filldf.append(test)

method2_filldf.to_pickle('~/Documents/fillevents.pkl')    
method2_filldf.head()

In [ ]:
# queryTEMPLATE = """
#     SELECT *
#     FROM ms_prod.smarts_raw_data
#     WHERE data_from_date = '{date}' and symbol = '{symbol}' and (event_date + event_time::time + event_millis::time) BETWEEN '{t1}' AND '{t2}' AND event_type in ('Place','Fill') 
#     """
                     
# method2_filldf = pd.DataFrame()
# for i in range(0,method2_datesymbol.shape[0]):
#     record = method2_datesymbol.iloc[i]
#     test = execute_query_data_frame(queryTEMPLATE.format(date = record['Date'].date(), symbol = record['symbol'], t1 = record['dt_min'], t2 = record['dt_max']), 'gemrdsdb', ssh = 'interim')     
#     print(i, test.shape)
#     method2_filldf = method2_filldf.append(test)

# method2_filldf.to_pickle('~/Documents/fillevents.pkl')    
# method2_filldf.head()

TypeError: execute_query_data_frame() missing 1 required positional argument: 'database'

In [ ]:
# ms_dev.daily_best_prices_split_by_account

In [ ]:
# queryTEMPLATE = """
#     SELECT dbp.*, srd.*
#     FROM ms_dev.daily_best_prices_split_by_account as dbp
#     LEFT JOIN ms_prod.smarts_raw_data as srd
#     ON dbp.event_id = srd.event_id
#     LIMIT 10;
#     """

In [27]:
# queryTEMPLATE = """
#     SELECT * from ms_prod.smarts_raw_data 
#     WHERE event_id = 7943527211
#     LIMIT 2;"""#.format(x = '2019-07-01')
# test = execute_query_data_frame(queryTEMPLATE, 'gemrdsdb', ssh = 'interim');test

In [ ]:
# queryTEMPLATE = """
#     SELECT tbd.*, dbp.*
#     FROM (SELECT * from ms_prod.smarts_raw_data WHERE data_from_date::date = '{x}' LIMIT 4) as tbd
#     LEFT JOIN 
#     (SELECT * from ms_dev.daily_best_prices_split_by_account where event_date::date = '{x}' LIMIT 3) as dbp
#     USING(event_id);
#     """.format(x = '2019-07-01')
# test = execute_query_data_frame(queryTEMPLATE, 'gemrdsdb', ssh = 'interim');test

In [ ]:
# SELECT tbd.*, dbp.*
#     FROM (SELECT * from ms_prod.smarts_raw_data WHERE data_from_date::date = '2019-07-01' LIMIT 4) as tbd
#     LEFT JOIN 
#     (SELECT * from ms_dev.daily_best_prices_split_by_account where event_date::date = '2019-07-01') as dbp
#     USING(event_id)